In [1]:
# Install necessary libraries
!pip install transformers datasets


In [5]:
# Importing necessary libraries
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import pandas as pd
import random

In [7]:
# Step 3: Load the Dataset
languages = ['en', 'es', 'fr', 'hi', 'sw', 'zh']  # High-resource and low-resource languages
datasets = {}

# Load the XNLI dataset for each language and store it in a dictionary
for lang in languages:
    datasets[lang] = load_dataset("xnli", lang)

README.md:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/50.2M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/157k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/310k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/157k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

In [18]:
# Convert the dataset to a Pandas DataFrame for easier manipulation
# Use select to take 100 samples from each language for simplicity
all_data = []
for lang in languages:
    dataset_samples = datasets[lang]['train'].select(range(100))
    for sample in dataset_samples:
        all_data.append(sample)

# Create a DataFrame from the collected data
data = pd.DataFrame(all_data)


In [27]:
# Step 4: Prepare Data for the Model
# Create a dictionary to hold queries (premise + hypothesis pairs) and their corresponding expected answers for each language. This dictionary will be used for model evaluation.

# Creating a dictionary for each language's queries and answers
queries_answers = {}
for lang in languages:
    # Filter the data for each language using the premise and hypothesis columns
    lang_data = data[data['premise'].str.contains(lang, na=False)]  # Adjust this if you have a better condition for language detection
    queries_answers[lang] = list( lang_data['hypothesis'] ,zip(lang_data['premise'] , lang_data['label']))

# Continue with Step 5 and beyond as usual


In [24]:
data

,premise,hypothesis,label
0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,1
1,you know during the season and i guess at at y...,You lose the things to the following level if ...,0
2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,0
3,How do you know ? All this is their informatio...,This information belongs to them .,0
4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices .,1
...,...,...,...
595,"点击 更 多 链接 ( 在 杂项 下 的 右手 边 ) , 以及",没有 链接 点击 杂项 .,2
596,"所以 我 开始 看 它 , 所有 的 突然 留下 关注 下周 , 我 去 了 什么 ,","如果 我 知道 , 我 就 不 会 开始 看 它 了",1
597,"哦 , 不 , 哦 , 好 吧 , 保重",再 见 了,0
598,""" 你好 , 本.''",我 忽略 了 本,2


In [29]:
# Step 5: Set Up Text Generation Pipeline
# Set up the LLaMA model and its tokenizer. We use a smaller version like "open_llama_3b" for computational efficiency.

# Load LLaMA model
model_name = "openlm-research/open_llama_3b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# Check if pad_token_id is missing, and set it to eos_token_id if needed
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Set up the text generation pipeline using LLaMA
generator = pipeline(
    "text-generation",
    model=model_name,
    tokenizer=tokenizer,
    return_full_text=False,
    torch_dtype="auto",  # Automatically select the best precision
    trust_remote_code=True,
    device_map="auto",
    temperature=0.2,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    no_repeat_ngram_size=3,
    max_new_tokens=150,
    do_sample=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,  # Prevent output looping
    output_attentions=False,
)



tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [36]:
### Step 6: Define Evaluation Function for Model
# We will now continue with evaluating the model using the corrected `queries_answers` dictionary.

def evaluate_model_on_queries(queries_answers, num_samples=10):
    # This function will evaluate the model on a sample of queries for each language.
    results = []

    for lang, queries in queries_answers.items():
        print(f"Evaluating language: {lang}")

        # Randomly sample a few queries from each language
        sample_queries = random.sample(queries, min(num_samples, len(queries)))

        for premise, hypothesis, expected_label in sample_queries:
            # Concatenate premise and hypothesis to create the input query
            query = f"Premise: {premise} Hypothesis: {hypothesis} Is the hypothesis true given the premise you have 3 options to answer?"

            # Generate a response from the model
            response = generator(query, max_new_tokens=50, num_return_sequences=1, truncation=True)[0]['generated_text']

            # Log the results including the original query, expected answer, and the response generated by the model
            results.append({
                "language": lang,
                "premise": premise,
                "hypothesis": hypothesis,
                "expected_label": expected_label,
                "response": response
            })

    # Convert results to a DataFrame for easy inspection
    return pd.DataFrame(results)


In [37]:
# Step 7: Perform Model Evaluation
# Evaluate the model on the sample queries from the dataset
evaluation_results = evaluate_model_on_queries(queries_answers)

Evaluating language: en


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Evaluating language: es
Evaluating language: fr
Evaluating language: hi
Evaluating language: sw
Evaluating language: zh


In [38]:
# Step 8: Save and Display Evaluation Results
# Save evaluation results to a CSV file and display the first few rows
evaluation_results.to_csv("multilingual_evaluation_results.csv", index=False)
print(evaluation_results.head())

  language                                            premise  \
0       en  Kuweka hatua kando kama kufikisha ya farasi , ...   
1       en  The poverty , for instance , does not create t...   
2       en                      Bonjour , ben . ' ' ' ' ' ' '   
3       en  Yeah i tell you what though ukienda bei baadhi...   
4       en  and so i started watching it and all of a sudd...   

                                          hypothesis  expected_label  \
0  Gail sheehy ni shabaha maarufu kwa masimango k...               1   
1  Poverty doesn 't create a sense of shame in an...               2   
2                                   J' ai ignoré ben               2   
3              Viatu vya tennis zina masafa ya bei .               1   
4  I wouldn 't have started watching it if I 'd k...               1   

                                            response  
0  \nThe purpose of this study is to find out whe...  
1  \nPremise: Poverty is a major cause of crime i...  
2  \nJ' ais

In [39]:
# Step 9: Error Analysis
# Perform basic error analysis to identify discrepancies between expected and generated answers.
def perform_error_analysis(evaluation_results):
    # Extract responses that do not match the expected label
    # Assuming we categorize the response manually into labels similar to expected labels (0, 1, 2)
    incorrect_results = evaluation_results[evaluation_results['response'] != evaluation_results['expected_label']]
    return incorrect_results

# Perform error analysis
errors = perform_error_analysis(evaluation_results)

# Save errors to CSV
errors.to_csv("error_analysis_results.csv", index=False)

# Display the first few rows of the error analysis results
print(errors.head())

  language                                            premise  \
0       en  Kuweka hatua kando kama kufikisha ya farasi , ...   
1       en  The poverty , for instance , does not create t...   
2       en                      Bonjour , ben . ' ' ' ' ' ' '   
3       en  Yeah i tell you what though ukienda bei baadhi...   
4       en  and so i started watching it and all of a sudd...   

                                          hypothesis  expected_label  \
0  Gail sheehy ni shabaha maarufu kwa masimango k...               1   
1  Poverty doesn 't create a sense of shame in an...               2   
2                                   J' ai ignoré ben               2   
3              Viatu vya tennis zina masafa ya bei .               1   
4  I wouldn 't have started watching it if I 'd k...               1   

                                            response  
0  \nThe purpose of this study is to find out whe...  
1  \nPremise: Poverty is a major cause of crime i...  
2  \nJ' ais

In [58]:
prefix = "you are an helpful assistant, i will give 2 sentences one is the Premise and the second one is Hypothesis. you need to decide about the relation between the Premise and Hypothesis you can choose in the output one of the three options: 1.condradiction , 2.Entailment 3. Neutral?"
query = f"Premise: i am happy  Hypothesis: i am sad Is the hypothesis true"
res = "response only in one of the following answers:  1.condradiction , 2.Entailment 3. Neutral?"
concat = prefix + "\n" + res+ "\n" + res+"\n" + res+"\n" + res+ "\n" +  query  
# Generate a response from the model
response = generator(query, max_new_tokens=50, num_return_sequences=1, truncation=True,temperature = 0.1)[0]['generated_text']

In [59]:
response

'?\nPremise: I am happy Hypotheses: I will be happy if I get a new car. I will not be happy If I don’t get a car.\nP1: I have a new house.\nH'

In [60]:
# Define the prefix and a set of multiple examples to fine-tune the model for better understanding
prefix = "You are a helpful assistant. I will give 2 sentences: one is the Premise and the second one is the Hypothesis. You need to decide on the relation between the Premise and Hypothesis. You can choose one of the following answers: 1. Contradiction, 2. Entailment, 3. Neutral."

# Adding multiple examples to help the model better understand the task
example_1 = "Premise: The sky is blue. Hypothesis: The sky is not blue."
example_2 = "Premise: John is in New York. Hypothesis: John is in the USA."
example_3 = "Premise: Mary likes coffee. Hypothesis: Mary dislikes coffee."
example_4 = "Premise: It is raining today. Hypothesis: It is sunny today."
example_5 = "Premise: All dogs are mammals. Hypothesis: Some dogs are not mammals."
example_6 = "Premise: The store opens at 9 AM. Hypothesis: The store is closed at 10 AM."
example_7 = "Premise: Tom went to the gym. Hypothesis: Tom worked out today."
example_8 = "Premise: Sarah finished her homework. Hypothesis: Sarah has completed her studies."
example_9 = "Premise: The book is on the table. Hypothesis: The book is in the drawer."
example_10 = "Premise: The car is red. Hypothesis: The vehicle is colored blue."

# Specify the response expectations to ensure the model stays within the three options
res = "Respond only with one of the following: 1. Contradiction, 2. Entailment, 3. Neutral."

# Concatenate the prefix, multiple examples, and the response instruction to create a complete prompt
concat = (
    f"{prefix}\n{res}\n"
    f"Example 1: {example_1}\n{res}\n"
    f"Example 2: {example_2}\n{res}\n"
    f"Example 3: {example_3}\n{res}\n"
    f"Example 4: {example_4}\n{res}\n"
    f"Example 5: {example_5}\n{res}\n"
    f"Example 6: {example_6}\n{res}\n"
    f"Example 7: {example_7}\n{res}\n"
    f"Example 8: {example_8}\n{res}\n"
    f"Example 9: {example_9}\n{res}\n"
    f"Example 10: {example_10}\n{res}\n"
)

# Example of the premise and hypothesis to be evaluated
query = "Premise: The sun rises in the east. Hypothesis: The sun rises in the west. Is the hypothesis true?"

# Concatenate the query with the previous context
concat += f"\n{query}"

# Generate a response from the model using the prompt
response = generator(concat, max_new_tokens=50, num_return_sequences=1, truncation=True, temperature=0.1)[0]['generated_text']

# Display the response
print(response)




A: The answer is "Contradiction".
The premise is true, but the hypothesis is false.

